In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


2.13.0-dev20230410


In [2]:
# get data files
x_train=pd.read_csv("train-data.tsv", sep='\t')
x_test=pd.read_csv("valid-data.tsv", sep='\t')

In [5]:
x_train.shape

(4178, 2)